In [230]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
import matplotlib.pyplot as plt

In [270]:
from string import Template

import numpy as np
import pandas as pd

import getpass
import json

from collections import defaultdict, namedtuple

from urllib.parse import quote_plus

from kgforge.core import KnowledgeGraphForge

from inference_tools.query.elastic_search import set_elastic_view
from inference_tools.utils import (check_premises,
                                   execute_query,
                                   execute_query_pipe,
                                   apply_rule)

In [233]:
TOKEN = getpass.getpass()

········


In [234]:
ENDPOINT = "https://staging.nexus.ocp.bbp.epfl.ch/v1"

In [235]:
forge = KnowledgeGraphForge(
    "../../configs/new-forge-config.yaml",
    endpoint=ENDPOINT,
    token=TOKEN, 
    bucket="dke/inference-test")

# Inference Rules

Get filters from atlas (or advanced query)

We get filters from atlas (or advanced query), for example:

    brainRegion "http://api.brain-map.org/api/v2/data/Structure/31"  # ACA
    species "http://purl.obolibrary.org/obo/NCBITaxon_10088"   # Mouse
    MType "http://uri.interlex.org/base/ilx_0381377"  # L6_UPC

In [237]:
input_filters = {
    "brainRegion": "http://api.brain-map.org/api/v2/data/Structure/31",
    "MType": "http://uri.interlex.org/base/ilx_0381377",
    "species": "http://purl.obolibrary.org/obo/NCBITaxon_10088",
}

Show all rules whose premises this search satisfies

- `KGQuery`: `SparqlQuery`, `ESQuery`, `ForgeSearchQuery`
- `RulePremise`: `PropertyPremise` 

Components of a `QueryBasedGeneralizationRule`:

- __Rule Parameters__ values from atlas filters or advanced query (species, brain region, cell type (m/e/t), datatype) 
- __Premises__ need to be satisfied on provided rule params for the rule to be applied
- __Query__ is applied to retrieve resources (result of inference). Queries can use the provided rule parameters

Course of actions:
    - Take filters
    - Take all the available rules (?)
    - Search for rules whose premises filters satisfy:
       -  For each rule execute 'premise query' filled with parameters generated from filters
      - Return rules whose premises are satisfied
    - Apply queries of satisfied rules

# Principle: PC cells in neocortex have the same E-behaviour

## Rule A (simple query)

Input parameters: brain_region, MType
If (premise)
   (brain region)-[is_part_of]->({id: isocortex})
   (mtype)-[IS_SUBCLASS_OF]->({id: PC})

Return X s.t.
- (X)-[IS_A]->(Trace)
- (X)-[subject/species]->({id: Mouse})
- (X)-[brainLocation/brainRegion]->(brain_region)-[is_part_of]->({id: isocortex})
- (X)-[brainLocation/layer]->(brain layer {id: L5})
- (X)-[hasMType]->(MType)-[IS_SUBCLASS_OF]->({id: PC})

In [375]:
premise = [
   {
        "type": "SparqlPremise",
        "hasParameter": [
            {
                "name": "brainRegion",
                "description": "URI of the query brain region",
                "type": "sparql_uri"
            }
        ],
        "hasBody": """
            SELECT ?x
            WHERE {
                <$brainRegion> <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
            }
        """,
        "queryConfiguration": {
            "org": "dke",
            "project": "inference-test",
            "sparqlView": {
                "id": "https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex",
                "type": "SparqlView"
            }
        }
    },
    {
        "type": "SparqlPremise",
        "hasParameter": [
            {
                "name": "MType",
                "description": "URI of the query MType",
                "type": "sparql_uri"
            }
        ],
        "hasBody": """
            SELECT ?x
            WHERE {
                <$MType> rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
            }
        """,
        "queryConfiguration": {
            "org": "dke",
            "project": "inference-test"
        }
    },
    {
        "type": "ForgeSearchPremise",
        "hasParameter": [
            {
                "name": "brainRegion",
                "description": "URI of the query brain region",
                "type": "sparql_uri"
            }
        ],
        "targetParameter": "brainRegion",
        "targetPath": "id",
        "pattern": {
            "isPartOf": "http://api.brain-map.org/api/v2/data/Structure/315"
        },
        "queryConfiguration": {
            "org": "dke",
            "project": "inference-test"
        }
    },
]

In [376]:
search_query_A = {
    "type": "SparqlQuery",
    "hasBody": """
        SELECT ?resource
        WHERE {
          ?resource a <https://neuroshapes.org/Trace> ;
                      <https://neuroshapes.org/annotation>/<https://neuroshapes.org/hasBody> ?mtype ;
                      <https://bluebrain.github.io/nexus/vocabulary/deprecated> false ;
                      <https://neuroshapes.org/brainLocation>/<https://neuroshapes.org/brainRegion> ?brainRegion.
          ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
          ?brainRegion <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
        }
    """,
    "queryConfiguration": {
        "org": "dke",
        "project": "inference-test"
    }
}

In [377]:
rule_A = {
    "type": "DataGeneralizationRule",
    "name": "Trace generalization for cortical PCs",
    "description": "Collect traces belonging to PCs from other cortical regions",
    "premise": premise,
    "searchQuery": search_query_A
}

In [378]:
%time
check_premises(rule_A, input_filters, TOKEN)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


True

In [379]:
%time
apply_rule(rule_A, input_filters, TOKEN)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


[{'resource': 'https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0'},
 {'resource': 'https://bbp.epfl.ch/neurosciencegraph/data/2b108ccf-4ad2-4495-8490-17b2fde656fa'},
 {'resource': 'https://bbp.epfl.ch/neurosciencegraph/data/4fba4ab3-189c-4ff4-96bf-6904c1b8912b'}]

In [380]:
# forge.register(forge.from_json(rule_A))

In [386]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/c6806339-a6fe-43d3-8d0c-6d9f68c90dfb"
rule_from_nexus = forge.as_json(forge.retrieve(rule_id))

In [387]:
check_premises(rule_from_nexus, input_filters, TOKEN)

True

In [389]:
apply_rule(rule_from_nexus, input_filters, TOKEN)

[{'id': 'https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0'},
 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/2b108ccf-4ad2-4495-8490-17b2fde656fa'},
 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/4fba4ab3-189c-4ff4-96bf-6904c1b8912b'}]

## Rule B (query + similarity search)

Input parameters: brain_region, MType, similarityModelId, topNeighbors

Find closest MType and then return traces/morphs with the closest MType that is also a subclass of pyramidal neuron and that are located in a cortical region.

If (premise)
   (brain region)-[is_part_of]->({id: isocortex})
   (mtype)-[IS_SUBCLASS_OF]->({id: PC})


Return Y s.t.
- (Y)-[hasMType]->(MTypeY)
- Select top K MTypeY by sim_{similarityModel}(MType, MTypeY) 

(or for traces)
Return Y s.t.
- (Y)-[hasEType]->(ETypeY)
- Select either top K ETypeY by sim_{similarityModel}(ETypeX, ETypeY) or s.t sim_{similarityModel}(ETypeX, ETypeY) > similarityThreshold

__Notes__:

- Similarity-based query is not allowed in premises
- You can use results from the previous query or input params in the next query

TODO: make rule know which view to use
- "configuration": "https://bbp.epfl.ch/neurosciencegraph/data/ontology-class-recommender-config",
- "similarityModels": "https://bbp.epfl.ch/neurosciencegraph/data/MTypeEmbeddingModel",          

In [390]:
search_query_B =  {
    "type": "QueryPipe",
    "head": {
        # Query type
        "type": "SparqlQuery",
        "queryConfiguration": {
            "org": "dke",
            "project": "inference-test",
            "sparqlView": {
                "id": "https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex",
                "type": "SparqlView"
            }
        },
        # Query body
        "hasBody": """
            SELECT ?mtype
            WHERE {
                ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
                ?trace <https://neuroshapes.org/annotation>/<https://neuroshapes.org/hasBody> ?mtype.
            }
        """,
        # Mapping of query results to the parameters (passed to the next query)
        "resultParameterMapping": [
            {
                "path": "mtype",
                "parameterName": "acceptableResults"
            }
        ]
    },
    "rest": { 
        "type": "QueryPipe",
        "head": {
            # Query type
            "type": "SimilarityQuery",
            # Number of top results to return
            "k": 1,
            "queryConfiguration": {
                "org": "dke",
                "project": "inference-test",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/MTypeEmbeddingModel",
                    "type": "EmbeddingModel"
                },
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/mtype-embedding-view",
                    "type": "ElasticSearchView"
                },
                "scoreFormula": "euclidean",
            },
            # Specification of input parameters
            "hasParameter": [
                {
                    "name": "MType",
                    "description": "URI of the query MType",
                    "type": "str"
                },
                {
                    "name": "acceptableResults",
                    "description": "List of URI's of acceptable MTypes (all PCs)",
                    "type": "list"
                }
            ],
            # Query to retrieve embeddings
            "searchTargetParameter": "MType",
            # Filter to apply on the similarity search results
            "resultFilter": """
                "must": {
                    "nested": {
                        "path": "derivation.entity",
                        "query": {
                            "terms": { "derivation.entity.@id": [$acceptableResults] }
                        }
                    }
                }
            """,
            # Mapping of query results to the parameters (passed to the next query)
            "resultParameterMapping": [{
                "path": "id",
                "parameterName": "closestMType"
            }]
        },
        "rest": {
            # Query type
            "type": "SparqlQuery",
            # Specification of input parameters
            "hasParameter": [
                {
                    "name": "closestMType",
                    "description": "URI of the closest acceptable MType",
                    "type": "sparql_uri"
                }
            ],
            "queryConfiguration": {
                "org": "dke",
                "project": "inference-test",
                "sparqlView": {
                    "id": "https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex",
                    "type": "SparqlView"
                }
            },
            # Query body
            "hasBody": """
                SELECT ?id
                WHERE {
                    ?id a <https://neuroshapes.org/Trace> ;
                        <https://bluebrain.github.io/nexus/vocabulary/deprecated> false ;
                        <https://neuroshapes.org/annotation> ?annotation ;
                        <https://neuroshapes.org/brainLocation>/<https://neuroshapes.org/brainRegion> ?brainRegion.
                    ?annotation <https://neuroshapes.org/hasBody> <$closestMType>.
                }
            """
        }
    }
}

In [391]:
rule_B = {
    "type": "DataGeneralizationRule",
    "name": "MType similarity based trace generalization for cortical PCs",
    "description": "Collect traces belonging to the closest (PC) MType from other cortical regions",
    "premise": premise,
    "searchQuery": search_query_B
}

In [392]:
check_premises(rule_B, input_filters, TOKEN)

True

In [393]:
%time
apply_rule(rule_B, input_filters, TOKEN)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


[{'id': 'https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0'}]

In [394]:
# forge.register(forge.from_json(rule_B))

In [395]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/1030e519-07c5-46ef-8e5c-7cca08c2f044"
rule_from_nexus = forge.as_json(forge.retrieve(rule_id))

In [397]:
apply_rule(rule_from_nexus, input_filters, TOKEN)

[{'id': 'https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0'}]

## Rule C

In [398]:
search_query_C = {
    "type": "QueryPipe",
    "head": {
        "type": "SparqlQuery",
        "queryConfiguration": {
            "org": "dke",
            "project": "inference-test",
            "sparqlView": {
                "id": "https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex",
                "type": "SparqlView"
            }
        },
        "hasBody": """
            SELECT ?mtype
            WHERE {
                ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
            }
        """,
        "resultParameterMapping": [{
            "path": "mtype",
            "parameterName": "acceptableMTypes"
        }]
    },
    "rest": {
        "type": "SimilarityQuery",
        "k": 1,
        # Configuration of similarity views
        # (multiple records mean that multiple models are combined)
        "queryConfiguration": [
            {
                "org": "dke",
                "project": "inference-test",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/MTypeEmbeddingModel",
                    "type": "EmbeddingModel"
                },
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/mtype-embedding-view",
                    "type": "ElasticSearchView"
                },
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/mtype-boosting-view",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/mtype-statistics-view",
                    "type": "ElasticSearchView"
                }
            },
            {
                "org": "dke",
                "project": "inference-test",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/MTypeEmbeddingModel2",
                    "type": "EmbeddingModel"
                },
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/mtype-embedding-view2",
                    "type": "ElasticSearchView"
                },
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/mtype-boosting-view2",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/mtype-statistics-view2",
                    "type": "ElasticSearchView"
                }
            }
        ],
        "hasParameter": [
            {
                "name": "MType",
                "description": "URI of the query MType",
                "type": "str"
            },
            {
                "name": "acceptableMTypes",
                "description": "URIs of acceptable MTypes (subclasses of PC).",
                "type": "list"
            }
        ],
        "searchTargetParameter": "MType",
        # Filter to apply on the similarity search results
        "resultFilter": """
            "must": {
                "nested": {
                    "path": "derivation.entity",
                    "query": {
                        "terms": { "derivation.entity.@id": [$acceptableMTypes] }
                    }
                }
            }
        """
    }
}

In [399]:
rule_C = {
    "@type": "DataGeneralizationRule",
    "name": "Combined (PC) MType similarity",
    "description": "Get the closest (PC) MType according to a combined similarity",
    "premise": [
        {
            "type": "SparqlPremise",
            "hasParameter": [
                {
                    "name": "MType",
                    "description": "URI of the query MType",
                    "type": "sparql_uri"
                }
            ],
            "hasBody": """
                SELECT ?x
                WHERE {
                    <$MType> rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
                }
            """,
            "queryConfiguration": {
                "org": "dke",
                "project": "inference-test",
                "sparqlView": {
                    "id": "https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex",
                    "type": "SparqlView"
                }
            }
        }
    ],
    "searchQuery": search_query_C
}

In [400]:
apply_rule(rule_C, input_filters, TOKEN)

[{'id': 'http://uri.interlex.org/base/ilx_0381371'}]

In [402]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/f7fced49-be40-4527-99fd-6c2d20615ce2"
rule_from_nexus = forge.as_json(forge.retrieve(rule_id))

In [404]:
apply_rule(rule_from_nexus, input_filters, TOKEN)

[{'id': 'http://uri.interlex.org/base/ilx_0381371'}]

# Get all the rules and check with ones can be applied

In [405]:
set_elastic_view(forge, "https://bbp.epfl.ch/neurosciencegraph/data/rule-view")

Get all the rules

In [406]:
all_rules = forge.elastic("""
    {
      "query": {
        "term": {
          "_deprecated": false
        }
      }
    }
""")

In [407]:
for r in all_rules:
    print(r.name)

Trace generalization for cortical PCs
MType similarity-based trace generalization for cortical PCs
Combined (PC) MType similarity


Get rules for target resource type `Trace`

In [408]:
trace_rules = forge.elastic("""{
  "query": {
    "bool": {
        "must": [
            {
               "term": {"targetResourceType": "Trace"}
            },
            {
                "term": {"_deprecated": false}
            }
        ]
     }
   }
}""")

In [409]:
for r in trace_rules:
    print(r.name)

Trace generalization for cortical PCs
MType similarity-based trace generalization for cortical PCs


In [411]:
for r in trace_rules:
    print(r.name)
    rule = forge.as_json(forge.retrieve(forge.as_json(r)["@id"]))
    
    params = {"MType": "bla", "brainRegion": "Bla"}

    print("\tPremise satisfied: ", check_premises(rule, params, TOKEN))
    result = apply_rule(rule, params, TOKEN)
    print("\tSearch results: ", result)

Trace generalization for cortical PCs
	Premise satisfied:  False
	Search results:  None
MType similarity-based trace generalization for cortical PCs


/Users/oshurko/Workspace/kg-inference/inference_tools/utils.py:280: InferenceToolsWarning: Rule premise is not satisfied on the input parameters
  def apply_rule(rule, parameters, token):


	Premise satisfied:  False
	Search results:  None


In [412]:
for r in trace_rules:
    print(r.name)
    rule = forge.as_json(forge.retrieve(forge.as_json(r)["@id"]))

    params = {"MType": "bla"}

    print("\tPremise satisfied: ", check_premises(rule, params, TOKEN))
    result = apply_rule(rule, params, TOKEN)
    print("\tSearch results: ", result)
    print()

Trace generalization for cortical PCs
	Premise satisfied:  False
	Search results:  None

MType similarity-based trace generalization for cortical PCs
	Premise satisfied:  False
	Search results:  None



/Users/oshurko/Workspace/kg-inference/inference_tools/utils.py:119: MissingParameterWarning: Premise is not satified, one or more parameters are missing. See the following exception: Parameter value 'brainRegion' is not specified
  for premise in rule["premise"]:


In [413]:
for r in trace_rules:
    print(r.name)
    rule = forge.as_json(forge.retrieve(forge.as_json(r)["@id"]))
    
    params = input_filters
    
    print("\tPremise satisfied: ", check_premises(rule, params, TOKEN))
    result = apply_rule(rule, params, TOKEN)
    print("\tSearch results: ")
    if isinstance(result, dict):
        result = [result]
    for el in result:
        print("\t\t", el["id"])
    print()

Trace generalization for cortical PCs
	Premise satisfied:  True
	Search results: 
		 https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0
		 https://bbp.epfl.ch/neurosciencegraph/data/2b108ccf-4ad2-4495-8490-17b2fde656fa
		 https://bbp.epfl.ch/neurosciencegraph/data/4fba4ab3-189c-4ff4-96bf-6904c1b8912b

MType similarity-based trace generalization for cortical PCs
	Premise satisfied:  True
	Search results: 
		 https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0



In [414]:
for r in all_rules:
    print(r.name)
    rule = forge.as_json(forge.retrieve(forge.as_json(r)["@id"]))
    
    params = input_filters
    
    print("\tPremise satisfied: ", check_premises(rule, params, TOKEN))
    result = apply_rule(rule, params, TOKEN)
    print("\tSearch results: ")
    if isinstance(result, dict):
        result = [result]
    for el in result:
        print("\t\t", el["id"])
    print()

Trace generalization for cortical PCs
	Premise satisfied:  True
	Search results: 
		 https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0
		 https://bbp.epfl.ch/neurosciencegraph/data/2b108ccf-4ad2-4495-8490-17b2fde656fa
		 https://bbp.epfl.ch/neurosciencegraph/data/4fba4ab3-189c-4ff4-96bf-6904c1b8912b

MType similarity-based trace generalization for cortical PCs
	Premise satisfied:  True
	Search results: 
		 https://bbp.epfl.ch/neurosciencegraph/data/ff0e496d-b8a0-4c0e-8487-545563495dc0

Combined (PC) MType similarity
	Premise satisfied:  True
	Search results: 
		 http://uri.interlex.org/base/ilx_0381371

